In [10]:
from util.backtest import *
from util.factor_util import *
import matplotlib.pyplot as plt
from matplotlib import cm
import itertools
from tqdm import tqdm
import warnings
import requests
from bs4 import BeautifulSoup
import glob

warnings.filterwarnings("ignore")

plt.style.use('seaborn')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
plt.rcParams['axes.unicode_minus'] =False

In [34]:
url = 'https://api.tardis.dev/v1/exchanges/binance-futures'
result = requests.get(url).json()
result['availableSymbols']
# perpetual_symbols = [sym['id'] for sym in result['availableSymbols'] if 'usdt' in sym['id'] and sym['type']=='perpetual']
perpetual_symbols = [sym['id'].replace('1000','').replace('usdt','') for sym in result['availableSymbols'] if 'usdt' in sym['id'] and sym['type']=='perpetual']
perpetual_symbols[:5]


['btc', 'eth', 'bch', 'xrp', 'eos']

## Binance Categories

In [12]:
bn_syms = requests.get('https://fapi.binance.com/fapi/v1/exchangeInfo').json()['symbols']
bn_syms = [sym for sym in bn_syms if 'USDT' in sym['symbol']]
bn_syms[0]


{'symbol': 'BTCUSDT',
 'pair': 'BTCUSDT',
 'contractType': 'PERPETUAL',
 'deliveryDate': 4133404800000,
 'onboardDate': 1569398400000,
 'status': 'TRADING',
 'maintMarginPercent': '2.5000',
 'requiredMarginPercent': '5.0000',
 'baseAsset': 'BTC',
 'quoteAsset': 'USDT',
 'marginAsset': 'USDT',
 'pricePrecision': 2,
 'quantityPrecision': 3,
 'baseAssetPrecision': 8,
 'quotePrecision': 8,
 'underlyingType': 'COIN',
 'underlyingSubType': ['PoW'],
 'settlePlan': 0,
 'triggerProtect': '0.0500',
 'liquidationFee': '0.012500',
 'marketTakeBound': '0.05',
 'filters': [{'minPrice': '556.80',
   'maxPrice': '4529764',
   'filterType': 'PRICE_FILTER',
   'tickSize': '0.10'},
  {'stepSize': '0.001',
   'filterType': 'LOT_SIZE',
   'maxQty': '1000',
   'minQty': '0.001'},
  {'stepSize': '0.001',
   'filterType': 'MARKET_LOT_SIZE',
   'maxQty': '120',
   'minQty': '0.001'},
  {'limit': 200, 'filterType': 'MAX_NUM_ORDERS'},
  {'limit': 10, 'filterType': 'MAX_NUM_ALGO_ORDERS'},
  {'notional': '5', 'fil

In [27]:
categories = {}
files = glob.glob('./data/html/*.html')
for file in files:
    with open(file) as f:
        result = f.read()
    soup = BeautifulSoup(result)
    contents = soup.find_all(class_='css-y492if')#css-1rdas1d # y492if
    contents = [content.getText().lower() for content in contents]
    category = file.split('/')[-1].split('_')[0].lower()
    if category not in categories.keys():
        categories[category] = []
    categories[category] += contents
    # categories[category] += list(set(contents) & set(perpetual_symbols))
    # print(category)
# len(contents)
categories.keys()

dict_keys(['defi', 'pos', 'innovation'])

In [28]:
links = ['Gaming','Metaverse','Layer1_Layer2','fan_token','NFT','storage-zone','Polkadot','pow','Launchpad','Launchpool','Infrastructure']

for link in tqdm(links):
    result = requests.get(f'https://www.binance.com/en/markets/coinInfo-{link}')
    soup = BeautifulSoup(result.text)
    contents = soup.find_all(class_='css-y492if')#css-1rdas1d # y492if
    contents = [content.getText().lower() for content in contents]
    
    category = link.lower()
    if category not in categories.keys():
        categories[category] = []
    categories[category] += contents    
    # categories[category] += list(set(contents) & set(perpetual_symbols))
# len(contents)
contents[:5]

100%|██████████| 11/11 [00:06<00:00,  1.66it/s]


['link', 'grt', 'bat', 'rndr', 'hnt']

In [29]:
for category in categories.keys():
    print(category,len(categories[category]))
with open('./data/Crytpo_binance_categories.json','w') as f:
    data = json.dumps(categories)
    f.write(data)

defi 110
pos 76
innovation 76
gaming 25
metaverse 11
layer1_layer2 44
fan_token 13
nft 28
storage-zone 12
polkadot 10
pow 31
launchpad 28
launchpool 30
infrastructure 25


In [ ]:
## BAPI Use for check
ls = [] 
for i in range(1,2001,100):
    binance_result = requests.get(f'https://www.binance.com/bapi/composite/v1/public/promo/cmc/cryptocurrency/listings/latest?limit=100&start={i}').json()['data']['body']['data']
    for data in binance_result:
        ls.append(data['symbol'])
        if data['symbol'] == 'ATOM':
            print(data)
            print()
    
    # print(i)
    # break


## CMC Categories

In [3]:
symbols = [sym[:-4].replace('1000','').replace('luna2','luna') for sym in perpetual_symbols]
symbols.remove('bttc')
symbols.remove('doteco')
symbols.remove('iota')

tmp = ','.join(symbols)
# 'bttc' in symbols

In [4]:
# requests.get('https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing?start=1&limit=100&sortBy=market_cap&sortType=desc&convert=USD,BTC,ETH&cryptoType=all&tagType=all&audited=false&aux=ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,tags,platform,max_supply,circulating_supply,self_reported_circulating_supply,self_reported_market_cap,total_supply,volume_7d,volume_30d&tagSlugs=polychain-capital-portfolio').json()['data']['cryptoCurrencyList']
KEY = '5a9640e1-d3a9-42ab-8c41-87de17934206'
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '5a9640e1-d3a9-42ab-8c41-87de17934206',
}
result = requests.get(f'https://pro-api.coinmarketcap.com/v2/cryptocurrency/info?symbol={tmp}',headers=headers).json()


In [5]:
result['data']['ETH'][0]#.keys()

{'id': 1027,
 'name': 'Ethereum',
 'symbol': 'ETH',
 'category': 'coin',
 'description': 'Ethereum (ETH) is a cryptocurrency . Ethereum has a current supply of 122,373,866.2178. The last known price of Ethereum is 1,664.30449268 USD and is up 1.30 over the last 24 hours. It is currently trading on 6492 active market(s) with $9,139,054,398.67 traded over the last 24 hours. More information can be found at https://www.ethereum.org/.',
 'slug': 'ethereum',
 'logo': 'https://s2.coinmarketcap.com/static/img/coins/64x64/1027.png',
 'subreddit': 'ethereum',
 'notice': '',
 'tags': ['pos',
  'smart-contracts',
  'ethereum-ecosystem',
  'coinbase-ventures-portfolio',
  'three-arrows-capital-portfolio',
  'polychain-capital-portfolio',
  'binance-labs-portfolio',
  'blockchain-capital-portfolio',
  'boostvc-portfolio',
  'cms-holdings-portfolio',
  'dcg-portfolio',
  'dragonfly-capital-portfolio',
  'electric-capital-portfolio',
  'fabric-ventures-portfolio',
  'framework-ventures-portfolio',
  

In [6]:
cmc_info = {}
keys = ['category', 'platform', 'tags', 'tag-names', 'tag-groups']

for key in result['data'].keys():
    data = result['data'][key][0]
    # platform = ''
    # if data['platform']:
    #     platform = data['platform']['name']
    #     print(data['symbol'],data['platform']['name'])
    
    cmc_info[data['symbol'].lower()] = {
        'id':data['id'],
        'name':data['name'],
        'slug':data['slug'],
        'category':list(np.unique(data['category'])),
        'platform':data['platform']['name'] if data['platform'] else None,
        'tags':list(np.unique(data['tags'])),
        'tag-names':list(np.unique(data['tag-names'])),
        'tag-groups':list(np.unique(data['tag-groups'])),
    }

        


In [9]:
with open('./data/Crytpo_cmc_info.json','w') as f:
    data = json.dumps(cmc_info)
    f.write(data)

In [42]:
tags = []
types = []
platforms = []
for key,value in cmc_info.items():
    if value['platform']:
        platforms.append(value['platform'])
        # print(key,value['platform'])
    for tag in value['tags']:
        tags.append(tag)
    types += value['category']
tags = pd.Series(tags).unique()
platforms = pd.Series(platforms).unique()
types = list(np.unique(types))

In [43]:
category_tags = [
    'bnb-chain','defi','binance-smart-chain','binance-launchpad','binance-launchpool',
    'collectibles-nfts','centralized-exchange','derivatives', 'dex',
    'ethereum-ecosystem', 'layer-2',
    'gaming','polkadot-ecosystem',
    'solana-ecosystem',
    'token','cross-chain'
    'pos','pow'
]
categories = {category:[] for category in category_tags}
for platform in platforms:
    categories[platform] = []
for type in types:
    categories[type] = []


for key,value in cmc_info.items():    
    for tag in value['tags']:
        if tag in categories.keys():
            categories[tag].append(key)
    if value['platform']:
        categories[value['platform']].append(key)
    if value['category']:
        for type in value['category']:
            categories[type].append(key)
            


In [44]:
# categories['bnb-chain']
for key in categories.keys():
    print(key,len(categories[key]))

bnb-chain 43
defi 59
binance-smart-chain 4
binance-launchpad 17
binance-launchpool 8
collectibles-nfts 29
centralized-exchange 3
derivatives 6
dex 5
ethereum-ecosystem 16
layer-2 2
gaming 16
polkadot-ecosystem 15
solana-ecosystem 15
token 113
cross-chainpos 0
pow 12
Ethereum 100
BNB 21
Terra Classic 1
Aptos 1
Avalanche 2
Huobi Token 4
TRON 1
Celo 1
Neo 1
Klaytn 1
Optimism 1
Waves 1
Secret 1
coin 65


In [46]:
with open('./data/Crytpo_cmc_categories.json','w') as f:
    data = json.dumps(categories)
    f.write(data)

## Summation

In [43]:
with open('./data/Crytpo_binance_categories.json','r') as f:
    bn_categories = json.loads(f.read())

for type in bn_categories.keys():
    bn_categories[type] = list(set(bn_categories[type]) & set(perpetual_symbols))
    # for sym in bn_categories[type]:
    #     if sym+'usdt' not in perpetual_symbols:
    #         bn_categories[type].remove(sym)


with open('./data/Crytpo_cmc_categories.json','r') as f:
    cmc_categories = json.loads(f.read())
        

In [42]:
perpetual_symbols[:5]

['btc', 'eth', 'bch', 'xrp', 'eos']

In [44]:
for category in bn_categories.keys():
    print(category,len(bn_categories[category]))
# bn_categories['gaming']
# cmc_categories['solana-ecosystem']

defi 46
pos 47
innovation 24
gaming 12
metaverse 6
layer1_layer2 35
fan_token 1
nft 19
storage-zone 8
polkadot 7
pow 20
launchpad 17
launchpool 16
infrastructure 16


In [47]:
crytpo_categories = {
    'token':cmc_categories['token'],
    'coin':cmc_categories['coin'],
    
    'Ethereum':list(np.unique(cmc_categories['Ethereum'] + ['eth'])),
    'Solana-ecosystem':list(np.unique(cmc_categories['solana-ecosystem'] + ['sol'])),
    'bnb-chain':list(np.unique(cmc_categories['bnb-chain'] + ['bnb'])),
    'Polkadot':list(np.unique(cmc_categories['polkadot-ecosystem'] + bn_categories['polkadot'] + ['dot'])),

    'defi':list(np.unique(cmc_categories['defi'] + bn_categories['defi'])),
    'innovation':bn_categories['innovation'],
    'gaming':list(np.unique(cmc_categories['gaming'] + bn_categories['gaming'])),
    'nft':list(np.unique(cmc_categories['collectibles-nfts'] + bn_categories['nft'])),
    'infrastructure':bn_categories['infrastructure'],
    'layer1_layer2':bn_categories['layer1_layer2'],
    
    'pos':list(np.unique( bn_categories['pos'])),
    'pow':list(np.unique(cmc_categories['pow'] + bn_categories['pow'])),
}
for category in crytpo_categories.keys():
    print(category,len(crytpo_categories[category]))

with open('./data/Crytpo_categories.json','w') as f:
    data = json.dumps(crytpo_categories)
    f.write(data)

token 113
coin 65
Ethereum 101
Solana-ecosystem 15
bnb-chain 43
Polkadot 17
defi 68
innovation 24
gaming 18
nft 33
infrastructure 16
layer1_layer2 35
pos 47
pow 20


In [38]:
perpetual_symbols

['btc',
 'eth',
 'bch',
 'xrp',
 'eos',
 'ltc',
 'trx',
 'etc',
 'link',
 'xlm',
 'ada',
 'xmr',
 'dash',
 'zec',
 'xtz',
 'atom',
 'bnb',
 'ont',
 'iota',
 'bat',
 'vet',
 'neo',
 'qtum',
 'iost',
 'theta',
 'algo',
 'zil',
 'knc',
 'zrx',
 'comp',
 'omg',
 'doge',
 'sxp',
 'kava',
 'band',
 'rlc',
 'waves',
 'mkr',
 'snx',
 'dot',
 'defi',
 'yfi',
 'bal',
 'crv',
 'trb',
 'sushi',
 'rune',
 'sol',
 'egld',
 'icx',
 'storj',
 'blz',
 'uni',
 'avax',
 'ftm',
 'hnt',
 'enj',
 'flm',
 'tomo',
 'ren',
 'ksm',
 'near',
 'fil',
 'aave',
 'rsr',
 'lrc',
 'matic',
 'ocean',
 'bel',
 'ctk',
 'axs',
 'alpha',
 'zen',
 'skl',
 'grt',
 '1inch',
 'chz',
 'sand',
 'ankr',
 'lit',
 'unfi',
 'reef',
 'rvn',
 'sfp',
 'xem',
 'coti',
 'chr',
 'mana',
 'alice',
 'hbar',
 'one',
 'lina',
 'stmx',
 'dent',
 'celr',
 'hot',
 'mtl',
 'ogn',
 'nkn',
 'dgb',
 'shib',
 'icp',
 'bake',
 'gtc',
 'btcdom',
 'iotx',
 'audio',
 'c98',
 'mask',
 'ata',
 'dydx',
 'xec',
 'gala',
 'celo',
 'ar',
 'klay',
 'arpa',
 'ct

In [57]:
tmp = list(set(bn_categories['defi']) & set(cmc_categories['defi']))
ls = []
ls = [sym for sym in bn_categories['defi'] if sym not in tmp]
ls

['klay',
 'rif',
 'luna',
 'df',
 'c98',
 'quick',
 'woo',
 'idex',
 'reef',
 'lit',
 'celo',
 'spell',
 'chess',
 'om',
 'ghst',
 'nmr',
 'perp',
 'fis',
 'forth',
 'anc',
 'joe',
 'super',
 'pros',
 'wing',
 'multi',
 'auto',
 'dego',
 'alcx',
 'front',
 'torn',
 'xvs',
 'farm',
 'dexe',
 'pnt',
 'cream',
 'ast']